In [1]:
# Get credentials
from IPython.utils import io
with io.capture_output() as captured:
    %run ../Introduction.ipynb
    
# Instantiate client
from sentenai import Client, Event
sentenai = Client(host, auth)

from datetime import datetime, timedelta

# Event Streams

Within the event stream API, the collection of event streams is represented as a dictionary-like Streams object with each stream name as the key.

In [7]:
dict(sentenai.streams)

{'1-14-2013_metlog': Stream(name='1-14-2013_metlog'),
 '1-16-2013_P-MALUS_Met_Log': Stream(name='1-16-2013_P-MALUS_Met_Log'),
 '1-16-2013_P-MALUS_Met_Log2': Stream(name='1-16-2013_P-MALUS_Met_Log2'),
 '1-16-2013_P-MALUS_Met_Log3': Stream(name='1-16-2013_P-MALUS_Met_Log3'),
 '20130123_MetLog_SMEOS_IndiaTowerTestTA1': Stream(name='20130123_MetLog_SMEOS_IndiaTowerTestTA1'),
 '20130124_MetLog_SMEOS_IndiaTowerTestTA1': Stream(name='20130124_MetLog_SMEOS_IndiaTowerTestTA1'),
 '20130128_MetLog_SMEOS_IndiaTowerTestTA1': Stream(name='20130128_MetLog_SMEOS_IndiaTowerTestTA1'),
 '20130201_MetLog_SMEOS_IndiaTowerTestTA1': Stream(name='20130201_MetLog_SMEOS_IndiaTowerTestTA1'),
 '2e20e6523a94418b8b13acbf32f9ac20': Stream(name='2e20e6523a94418b8b13acbf32f9ac20'),
 'MetStation': Stream(name='MetStation'),
 'MyDumbTestMetEvent_2019-07-01': Stream(name='MyDumbTestMetEvent_2019-07-01'),
 'PA0103atmoD139-193224': Stream(name='PA0103atmoD139-193224'),
 'PA0103atmoD142-101118': Stream(name='PA0103atmoD142-

---
### Streams can be accessed by their unique name
You may perform `getitem` on the streams API to instantiate a new Stream object. The stream may or may not exist. Referencing a new stream does not create a new event stream.

#### Getting a `Stream` object with `getitem`

In [4]:
weather = sentenai.streams['weather']
imaginary = sentenai.streams['imaginary']

In [13]:
weather['icon'] @ datetime.utcnow()

'partly-cloudy-night'

#### Streams may or may not exist

A Stream that does not exist is one that has not been created server-side, even if you have a client object.

In [97]:
print("weather exists? {}".format(bool(weather)))
print("imaginary exists? {}".format(bool(imaginary)))

weather exists? True
imaginary exists? False


#### Streams may be filtered to create sub-streams

In [98]:
weather = sentenai.streams['weather']
cloudy_weather = weather.where('icon = "cloudy"')

In [99]:
len(cloudy_weather.events)

4

---
### Creating Streams

Creating streams can be done via their `.init()` method. When initializing a new stream, it takes an optional parameter `t0` which indicates what point in time to treat as `0`. By default, the creation time of the stream will be taken as `t0`, but you might want to choose `datetime(1970,1,1)` (Unix epoch) or `None` (stream not anchored to real time) instead.

In [12]:
sentenai.streams['temp'].init(t0=None) # virtual time
sentenai.streams['temp_real'].init(t0=datetime(1970,1,1)) # Unix Epoch
sentenai.streams['temp_now'].init() # Current UTC Time.

**Important note**: In the past, creating a stream could be done via metadata or by inserting events. The addition of per-stream t0 requires eliminating implicit stream creation and requires this instantiation of streams.

---
### Deleting streams

Deleting a stream eliminates all information about a stream along with all events that are a member of that stream.

This operation may take a second.

In [100]:
try:
  del sentenai.streams['imaginary']
except:
    pass

## Stream Metadata

Event Streams may have user-set metadata, which can be accessed via the `.metadata` sub-object.


---
### Setting metadata

In [101]:
weather.metadata['flag'] = True
weather.metadata['str'] = "hello world"
weather.metadata['count'] = 4

---
### Getting metadata

In [105]:
dict(weather.metadata)

{'count': 4, 'flag': True, 'str': 'hello world'}

In [12]:
weather.metadata['flag']

True

---
### Deleting metadata

In [13]:
del weather.metadata['flag']

#### Clear metadata

In [14]:
del weather.metadata
dict(weather.metadata)

{'count': 4, 'flag': True, 'str': 'hello world'}

## Core Stream methods

Stream objects have a number of built in properties and methods.

---
### Adding events to a stream

The `Stream` class provides three methods for managing events: `insert`, `update`, and `remove`.

#### Inserting a new event

In [15]:
test = sentenai.streams['test-stream']

my_evt = test.insert(Event(ts=datetime.utcnow(), data={'temperatureMax': -500.0}))
print(my_evt)

Event(id='nfnCiJPHlkHBcl89y5FpY-0l', ts=numpy.datetime64('2019-07-25T18:31:57.408215'), data={'temperatureMax': -500.0})


#### Updating an event

In [16]:
my_evt.data = {'temperatureMax': -50.0}
test.update(my_evt)
test[my_evt.id]

Stream(name='test-stream')["nfnCiJPHlkHBcl89y5FpY-0l"]

#### Deleting an event

In [17]:
del test.events[my_evt.id] # or weather.remove(evt.id)

try:
    test.events[my_evt.id]
except KeyError:
    print("Event successfully deleted")

Event successfully deleted


## The `Stream.events` collection

All events belonging to a stream are stored in the stream's events collection.

---
### A stream's events collection is list-like and time-ordered.

Every stream contains a collection of events. The event sub-API is located at `Stream.events`.


In [6]:
ostream = sentenai.streams['ostream']
for i in range(20):
    ostream.insert(Event(ts=datetime(2018,1,1)+timedelta(seconds=60*i), data={'a': i / 2, 'b': i * 3}))

#### List operations

In [7]:
len(ostream.events)

20

In [8]:
ostream.events[0]

Event(id='X-p8KBLtT_ZUwS-Q_xYi8grp', ts=numpy.datetime64('2018-01-01T00:00:00'), data={'a': 0.0, 'b': 0})

In [9]:
ostream.events[5]

Event(id='29CX9dBBcSeYzTspCmJMik3H', ts=numpy.datetime64('2018-01-01T00:05:00'), data={'a': 2.5, 'b': 15})

In [10]:
ostream.events[-1]

Event(id='5MC8urMgvwlEbbfrqXu8-I99', ts=numpy.datetime64('2018-01-01T00:19:00'), data={'a': 9.5, 'b': 57})

In [11]:
for evt in ostream.events[3:7]:
    print(evt.id, evt.ts)

_opeyjlFiM6R8Ksj3paGLPQs 2018-01-01T00:03:00
y8DefihIXu9yr1SGpCpUBS1N 2018-01-01T00:04:00
29CX9dBBcSeYzTspCmJMik3H 2018-01-01T00:05:00
7-O4g5mxcFOYvKyLj3jnYpxx 2018-01-01T00:06:00


#### A Stream's events collection can be sliced by time.

In [12]:
for evt in ostream.events[datetime(2018,1,1,0,5):datetime(2018,1,1,0,10)]:
    print(evt.id, evt.ts)

29CX9dBBcSeYzTspCmJMik3H 2018-01-01T00:05:00
7-O4g5mxcFOYvKyLj3jnYpxx 2018-01-01T00:06:00
w7fXH1bGqP6uev4trCYNAX4k 2018-01-01T00:07:00
VBIdcLq7xnHSKl6aLdRyPD1G 2018-01-01T00:08:00
p8cMTMALJJDiVVYrl3HNwITA 2018-01-01T00:09:00


#### Time slices can use a limit to prevent returning too many events

In [13]:
for evt in ostream.events[datetime(2018,1,1,0,5):datetime(2018,1,1,0,10):2]:
    print(evt.id, evt.ts)

29CX9dBBcSeYzTspCmJMik3H 2018-01-01T00:05:00
7-O4g5mxcFOYvKyLj3jnYpxx 2018-01-01T00:06:00


---
### A stream's events collection is dict-like

In [26]:
ostream.events['ABCD'] = Event(data={'foo': False})

In [27]:
ostream.events['ABCD']

Event(id='ABCD', ts=numpy.datetime64('2019-07-25T18:32:14.111867434'), data={'foo': False})

In [28]:
del ostream.events['ABCD']

## Stream schemas

Since events streams are a time-ordered collection of events, and events are represented as a tree of values and subtrees (think JSON-like objects), we can represent a `Stream`'s schema as a dictionary of paths to each value, also known as a `Field`.

---
### Stream Fields

In [29]:
stream1 = sentenai.streams['stream1']

deep_evt = Event(
    ts=datetime(2015,1,1,1,1),
    data={
        'a': {
            'aa': 1,
            'bb': {
                'cc': 8,
                'dd': "yes"
            }
        },
        'b': False
    }
)

stream1.events.insert(deep_evt)

for field in stream1:
    print(field.path)

('a', 'aa')
('a', 'bb', 'cc')
('a', 'bb', 'dd')
('b',)


##### To access a field, use dictionary-like notation:

In [30]:
stream1['a', 'bb', 'cc']

Stream(name='stream1')["a", "bb", "cc"]

---
### Fields statistics

#### Global statistics

In [31]:
weather['humidity'].count, weather['humidity'].missing

(345, 0)

In [32]:
weather['humidity'].mean

0.6473913043478261

In [33]:
weather['humidity'].min, weather['humidity'].max

(0.28, 0.98)

In [34]:
weather['humidity'].std

0.15552099400186561

In [35]:
weather['icon'].unique

{'partly-cloudy-day': 163,
 'rain': 94,
 'partly-cloudy-night': 48,
 'clear-day': 21,
 'snow': 6,
 'fog': 5,
 'cloudy': 4,
 'wind': 4}

In [36]:
weather['icon'].top

'partly-cloudy-day'

#### Local Statistics
Numerical statistics like `min`, `max`, `mean`,  and `std` can also be retrieved for a slice of time. Note that these stats treat all values with equal weight, they are not time-weighted at this time. That means they work best with fixed interval events, e.g. meteorological data.

In [37]:
weather['humidity'][datetime(2010,6,1):datetime(2010,7,1)]._stats

{'categorical': {},
 'numerical': {'max': 0.93,
  'mean': 0.6963333333333334,
  'count': 30,
  'missing': 0,
  'min': 0.4,
  'std': 0.13241739997482196}}

## Values

Streams aren't just collections of events; they also have encapsulate the temporal concept of having *values* at a point in time, where the collection of values at a point in time is the set of all paths paired with the value of that path when it was last set.


---
### Stream Values
The `stream.values(at=)` method returns the latest values at a point in time as a dictionary of paths to values. Values set in the past will be forward-filled to this point in time.

In [5]:
stream1 @ datetime(2015,1,1,1,1)      # or use stream1.values(at=...)

NameError: name 'stream1' is not defined

---
### Field Values
A field's value at a point in time can be accessed via the `@` operator.

In [39]:
weather['temperatureMax'] @ datetime(2010,6,1)     # or use weather['temperatureMax'].value(at=...)

83.42

----
### Time values (bounds)
The left bound of a stream is the timestamp of the oldest event in the stream. The right bound of a stream is the timestamp of the newest event in the stream.

In [40]:
weather.bounds

(numpy.datetime64('2010-01-01T00:00:00'),
 numpy.datetime64('2010-12-11T00:00:00'))